In [8]:
import numpy as np
import cvxpy as cp
import pandas as pd

# === Settings ===
def g(x):
    return np.array([1, x, x**2])

def lambda_fn(x):
    return 2 * x + 5
    # return 1  # for homoscedastic case

N = 201
x_vals = np.linspace(-1, 1, N)
p = 3

# Precompute g(x) and lambda(x)
g_list = [g(x) for x in x_vals]
lambda_vals = np.array([lambda_fn(x) for x in x_vals])

# === Precompute G_tensor for vectorized M ===
G_tensor = np.zeros((p, p, N))
for i in range(N):
    G_tensor[:, :, i] = lambda_vals[i] * np.outer(g_list[i], g_list[i])

# === Solve G-optimal design via CVXPY ===
w = cp.Variable(N)
t = cp.Variable()
M = cp.Variable((p, p), PSD=True)

constraints = [
    cp.sum(w) == 1,
    w >= 0,
    M >> 1e-6 * np.eye(p),
    M == cp.sum([w[i] * G_tensor[:, :, i] for i in range(N)])
]

# Use Schur complement to replace matrix_frac(g, M) <= t
for i in range(N):
    gx = g_list[i]
    Gx = gx.reshape(-1, 1)
    schur_mat = cp.bmat([
        [cp.reshape(t, (1, 1), order='F'), Gx.T],
        [Gx, M]
    ])
    constraints.append(schur_mat >> 0)

# Solve the problem
prob = cp.Problem(cp.Minimize(t), constraints)
if 'MOSEK' in cp.installed_solvers():
    prob.solve(solver=cp.MOSEK, verbose=False)
else:
    prob.solve(solver=cp.SCS, eps=1e-9, max_iters=50000, verbose=False)

w_val = w.value

# === Output support points ===
threshold = 1e-3
support_idx = np.where(w_val > threshold)[0]
x_out = np.round(x_vals[support_idx], 3)
w_out = np.round(w_val[support_idx], 3)

support_table = pd.DataFrame({'x': x_out, 'weight': w_out})
print("Support points and weights:")
print(support_table)

Support points and weights:
      x  weight
0 -1.00   0.495
1  0.07   0.293
2  1.00   0.212
